<a href="https://colab.research.google.com/github/kalai89730/project/blob/main/SQL_code_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch transformers bitsandbytes accelerate sqlparse

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.3 MB/s eta 0:00:00


In [3]:
import torch

In [4]:
from transformers import AutoTokenizer,AutoModelForCausalLM

In [5]:
torch.cuda.is_available()

True

In [6]:
available_memory=torch.cuda.get_device_properties(0).total_memory


In [7]:
available_memory

15835660288

In [10]:
model_name="defog/sqlcoder-7b-2"
tokenizer=AutoTokenizer.from_pretrained(model_name)
if available_memory>16e9:
  model=AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      torch_dtype=torch.float16,
      device_map="auto",
      use_cache=True
  )
else:
  model=AutoModelForCausalLM.from_pretrained(
      model_name,
      trust_remote_code=True,
      #torch_dtype=torch.float16,
      load_in_8bit=True,
      device_map="auto",
      use_cache=True
  )


config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [26]:
prompt="""### Task
generate a MYsql query to answer [QUESTION]{question}[/QUESTION].
###Instructions:
The input will be a natural language description of a SQL task.
You need to convert that description into a valid SQL query.
Make sure the SQL syntax is correct and optimized.
The SQL query should be based on common database operations like SELECT, INSERT, UPDATE, DELETE, JOIN, and GROUP BY.
If applicable, ensure the query includes conditions (WHERE), sorting (ORDER BY), limits (LIMIT), or any relevant clauses as required by the input.
Ensure to return only the SQL query (not additional explanations or comments).
if you can't answer the question within the available database schema return 'I DON;T KNOW '


#database schema
CREATE TABLE customers (
    customer_id INT PRIMARY KEY AUTO_INCREMENT,  -- Unique identifier for each customer
    first_name VARCHAR(50) NOT NULL,             -- First name of the customer
    last_name VARCHAR(50) NOT NULL,              -- Last name of the customer
    email VARCHAR(100) UNIQUE NOT NULL,          -- Email address, must be unique
    phone VARCHAR(20),                           -- Phone number (optional)
    address VARCHAR(255),                        -- Address (optional)
    city VARCHAR(100),                           -- City (optional)
    state VARCHAR(50),                           -- State (optional)
    zip_code VARCHAR(20),                        -- Zip code (optional)
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP  -- Timestamp of when the record is created
);

###answer
given the database schema here is the sql query that answer [QUESTION]{question}[/QUESTION] [SQL]
"""

In [27]:

import sqlparse
def generate_query(question):
  updated_prompt=prompt.format(question=question)
  inputs=tokenizer(updated_prompt,return_tensors="pt").to("cuda")
  generate_ids=model.generate(
      **inputs,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.pad_token_id,
      max_new_tokens=400,
      do_sample=False,
      num_beams=1,
  )
  outputs=tokenizer.batch_decode(generate_ids,skip_special_tokens=True)
  torch.cuda.empty_cache()
  torch.cuda.synchronize()
  return sqlparse.format(outputs[0].split("[SQL]")[-1],reindent=True)

In [28]:
question=input("Enter the query question : ")
generated_sql=generate_query(question)
generated_sql

Enter the query question : \nSELECT c.first_name,\n       c.email\nFROM customers c\nWHERE c.city ilike '%coimbatore%'\nORDER BY c.first_name DESC NULLS LAST\nLIMIT 5;


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


"\nSELECT c.first_name,\n       c.email\nFROM customers c\nWHERE c.city ilike '%coimbatore%'\nORDER BY c.first_name DESC NULLS LAST\nLIMIT 5;"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')